In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: VLA for Autonomous Port Terminal Tractors
## Implementation Notebook

Welcome to the implementation notebook for the VLA (Vision-Language-Action) case study. You will build a simplified VLA model that takes a port scene image and a dispatcher instruction, and generates a trajectory for an autonomous terminal tractor navigating a container shipping yard.

**What you will build:**
1. A synthetic port environment with container blocks, workers, and dispatcher instructions
2. A vision encoder (CNN) that processes port scene images
3. A text encoder (Transformer) that processes dispatcher instructions
4. A diffusion-based trajectory decoder that generates smooth, collision-aware paths
5. A complete training and evaluation pipeline

**Prerequisites:** PyTorch, NumPy, Matplotlib. All run on a free Colab T4 GPU.

**Estimated time:** 60-90 minutes.

---

## 1. Setup and Data Generation

### 1.1 Imports and Configuration

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/vla-autonomous-driving/practice/0/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from typing import Tuple, List, Dict
import random
import math
import time

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### 1.2 Port Environment

We model a container port yard as a 128x128 grid. Container blocks are arranged in rows with narrow corridors between them. Workers and temporary obstacles appear randomly. Each grid cell represents approximately 1.5 meters.

In [ ]:
# --- Port Environment Configuration ---
PORT_SIZE = 128          # Grid size (128x128 = ~200m x 200m at 1.5m/cell)
NUM_BLOCKS = 8           # Container blocks (A-H)
ROWS_PER_BLOCK = 12      # Rows per block
CORRIDOR_WIDTH = 2       # Grid cells between rows (~3 meters)
BLOCK_GAP = 6            # Grid cells between blocks (~9 meters)
NUM_WAYPOINTS = 32       # Trajectory length (3.2 seconds at 10Hz)
MAX_WORKERS = 4          # Max dynamic obstacles per scene
NUM_INSTRUCTION_TYPES = 4  # Types of dispatcher instructions


class PortEnvironment:
    """Generates synthetic port yard layouts with container blocks, corridors, and obstacles."""

    BLOCK_NAMES = [chr(65 + i) for i in range(NUM_BLOCKS)]  # A, B, C, ...

    def __init__(self, size: int = PORT_SIZE):
        self.size = size
        self.grid = np.zeros((size, size), dtype=np.float32)
        self.block_positions = {}  # block_name -> (row_start, col_start, row_end, col_end)
        self._build_layout()

    def _build_layout(self):
        """Place container blocks in a grid pattern."""
        block_width = 8   # cells wide per block
        block_height = ROWS_PER_BLOCK * (1 + CORRIDOR_WIDTH)

        start_x = 10
        start_y = 10
        col_idx = 0
        row_idx = 0

        for i, name in enumerate(self.BLOCK_NAMES):
            bx = start_x + col_idx * (block_width + BLOCK_GAP)
            by = start_y + row_idx * (block_height + BLOCK_GAP)

            if by + block_height >= self.size - 10:
                col_idx += 1
                row_idx = 0
                bx = start_x + col_idx * (block_width + BLOCK_GAP)
                by = start_y

            self.block_positions[name] = (by, bx, by + block_height, bx + block_width)

            # Fill container rows (leave corridors)
            for r in range(ROWS_PER_BLOCK):
                ry = by + r * (1 + CORRIDOR_WIDTH)
                self.grid[ry:ry+1, bx:bx+block_width] = 1.0  # container row

            row_idx += 1

    def get_destination(self, block_name: str, row: int) -> Tuple[int, int]:
        """Get the grid coordinates for a block/row destination."""
        if block_name not in self.block_positions:
            block_name = random.choice(list(self.block_positions.keys()))
        by, bx, _, bx_end = self.block_positions[block_name]
        row = min(row, ROWS_PER_BLOCK - 1)
        dest_y = by + row * (1 + CORRIDOR_WIDTH) + 1  # in the corridor
        dest_x = bx + (bx_end - bx) // 2  # center of block
        return (dest_y, dest_x)

    def add_random_workers(self, n_workers: int) -> List[Tuple[int, int]]:
        """Add random worker positions in corridors."""
        workers = []
        attempts = 0
        while len(workers) < n_workers and attempts < 100:
            y = random.randint(5, self.size - 5)
            x = random.randint(5, self.size - 5)
            if self.grid[y, x] == 0:  # only in free space
                workers.append((y, x))
            attempts += 1
        return workers

    def render(self, tractor_pos=None, trajectory=None, workers=None, destination=None):
        """Render the port as a 3-channel image (128x128x3)."""
        img = np.zeros((self.size, self.size, 3), dtype=np.float32)
        # Channel 0: container blocks (static obstacles)
        img[:, :, 0] = self.grid
        # Channel 1: dynamic obstacles (workers)
        if workers:
            for wy, wx in workers:
                y_lo, y_hi = max(0, wy-1), min(self.size, wy+2)
                x_lo, x_hi = max(0, wx-1), min(self.size, wx+2)
                img[y_lo:y_hi, x_lo:x_hi, 1] = 1.0
        # Channel 2: destination marker
        if destination:
            dy, dx = destination
            y_lo, y_hi = max(0, dy-2), min(self.size, dy+3)
            x_lo, x_hi = max(0, dx-2), min(self.size, dx+3)
            img[y_lo:y_hi, x_lo:x_hi, 2] = 1.0
        return img

### 1.3 Expert Trajectory Planner

The expert planner generates ground-truth trajectories using linear interpolation with noise and smoothing. In the real system, these would come from recorded human driver data.

In [ ]:
def plan_trajectory(grid, start, end, num_waypoints=NUM_WAYPOINTS):
    """
    Plan a trajectory from start to end, avoiding obstacles.
    Uses a simplified approach: straight-line with corridor-aware waypoint adjustment.
    Returns normalized waypoints in [-1, 1] relative to start position.
    """
    sy, sx = start
    ey, ex = end

    # Generate raw waypoints via linear interpolation
    raw_y = np.linspace(sy, ey, num_waypoints + 10)
    raw_x = np.linspace(sx, ex, num_waypoints + 10)

    # Add slight noise and smooth for realistic trajectories
    noise_scale = 0.3
    raw_y += np.random.randn(len(raw_y)) * noise_scale
    raw_x += np.random.randn(len(raw_x)) * noise_scale

    # Simple smoothing (moving average)
    kernel = np.ones(5) / 5
    raw_y = np.convolve(raw_y, kernel, mode='same')
    raw_x = np.convolve(raw_x, kernel, mode='same')

    # Subsample to desired number of waypoints
    indices = np.linspace(0, len(raw_y) - 1, num_waypoints).astype(int)
    traj_y = raw_y[indices]
    traj_x = raw_x[indices]

    # Normalize: convert to offsets from start, scale to [-1, 1] range
    traj_y = (traj_y - sy) / (PORT_SIZE / 2)
    traj_x = (traj_x - sx) / (PORT_SIZE / 2)

    # Compute heading at each waypoint
    dy = np.gradient(traj_y)
    dx = np.gradient(traj_x)
    theta = np.arctan2(dy, dx)

    trajectory = np.stack([traj_x, traj_y, theta], axis=-1).astype(np.float32)
    return trajectory  # shape: (NUM_WAYPOINTS, 3)

### 1.4 Dispatcher Instructions

In [ ]:
INSTRUCTION_TEMPLATES = [
    "Navigate to Block {block}, Row {row}.",
    "Pick up container at Block {block}, Row {row}. Proceed via main corridor.",
    "Deliver to Block {block}, Row {row}. Avoid Row {avoid_row} in Block {avoid_block} — maintenance in progress.",
    "Priority load. Go to Block {block}, Row {row} immediately.",
]

def generate_instruction(dest_block: str, dest_row: int, env: PortEnvironment) -> str:
    """Generate a natural language dispatcher instruction."""
    template_idx = random.randint(0, len(INSTRUCTION_TEMPLATES) - 1)
    template = INSTRUCTION_TEMPLATES[template_idx]

    avoid_block = random.choice(env.BLOCK_NAMES)
    avoid_row = random.randint(1, ROWS_PER_BLOCK)

    instruction = template.format(
        block=dest_block,
        row=dest_row,
        avoid_block=avoid_block,
        avoid_row=avoid_row,
    )
    return instruction

### 1.5 Dataset

In [ ]:
class PortVLADataset(Dataset):
    """
    Synthetic dataset for port VLA training.
    Each sample: (image, instruction_tokens, egomotion, trajectory)
    """

    def __init__(self, num_samples: int = 5000, max_instruction_len: int = 32):
        self.num_samples = num_samples
        self.max_instruction_len = max_instruction_len
        self.env = PortEnvironment()
        self.data = []

        # Build a simple word-to-index vocabulary from templates
        self.vocab = {"<pad>": 0, "<unk>": 1}
        all_words = set()
        for t in INSTRUCTION_TEMPLATES:
            for w in t.replace(".", "").replace(",", "").replace("{", "").replace("}", "").split():
                all_words.add(w.lower())
        for name in self.env.BLOCK_NAMES:
            all_words.add(name.lower())
        for i in range(1, ROWS_PER_BLOCK + 1):
            all_words.add(str(i))
        for i, w in enumerate(sorted(all_words), start=2):
            self.vocab[w] = i
        # Add special tokens for block names and numbers
        for c in "abcdefghijklmnopqrstuvwxyz":
            if c not in self.vocab:
                self.vocab[c] = len(self.vocab)

        self.vocab_size = len(self.vocab)
        self._generate_all()

    def tokenize(self, text: str) -> torch.Tensor:
        """Simple whitespace tokenizer."""
        words = text.replace(".", "").replace(",", "").replace("—", " ").lower().split()
        tokens = [self.vocab.get(w, 1) for w in words]  # 1 = <unk>
        # Pad or truncate
        if len(tokens) < self.max_instruction_len:
            tokens += [0] * (self.max_instruction_len - len(tokens))
        else:
            tokens = tokens[:self.max_instruction_len]
        return torch.tensor(tokens, dtype=torch.long)

    def _generate_all(self):
        """Pre-generate all samples."""
        for _ in range(self.num_samples):
            # Random start position (in a corridor)
            start_y = random.randint(15, PORT_SIZE - 15)
            start_x = random.randint(15, PORT_SIZE - 15)
            while self.env.grid[start_y, start_x] > 0:
                start_y = random.randint(15, PORT_SIZE - 15)
                start_x = random.randint(15, PORT_SIZE - 15)

            # Random destination
            dest_block = random.choice(self.env.BLOCK_NAMES)
            dest_row = random.randint(1, ROWS_PER_BLOCK)
            dest_pos = self.env.get_destination(dest_block, dest_row - 1)

            # Generate components
            workers = self.env.add_random_workers(random.randint(0, MAX_WORKERS))
            image = self.env.render(
                tractor_pos=(start_y, start_x),
                workers=workers,
                destination=dest_pos
            )
            instruction = generate_instruction(dest_block, dest_row, self.env)
            tokens = self.tokenize(instruction)
            trajectory = plan_trajectory(self.env.grid, (start_y, start_x), dest_pos)

            # Egomotion: simple straight-line history (16 waypoints, 3D)
            ego_history = np.zeros((16, 3), dtype=np.float32)
            ego_history[:, 0] = np.linspace(-0.1, 0, 16)  # slight forward motion

            self.data.append({
                "image": torch.tensor(image).permute(2, 0, 1),  # (3, 128, 128)
                "instruction": tokens,
                "egomotion": torch.tensor(ego_history),
                "trajectory": torch.tensor(trajectory),
                "start": (start_y, start_x),
                "dest": dest_pos,
                "workers": workers,
            })

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        d = self.data[idx]
        return d["image"], d["instruction"], d["egomotion"], d["trajectory"]


# Generate datasets
print("Generating training dataset (5000 samples)...")
train_dataset = PortVLADataset(num_samples=5000)
print("Generating validation dataset (500 samples)...")
val_dataset = PortVLADataset(num_samples=500)

print(f"\nVocabulary size: {train_dataset.vocab_size}")
print(f"Sample image shape: {train_dataset[0][0].shape}")
print(f"Sample instruction shape: {train_dataset[0][1].shape}")
print(f"Sample trajectory shape: {train_dataset[0][3].shape}")

### 1.6 TODO 1: Data Augmentation

Implement two data augmentation strategies to improve model robustness.

In [ ]:
def augment_trajectory_noise(trajectory: torch.Tensor, noise_std: float = 0.01) -> torch.Tensor:
    """
    Add small Gaussian noise to expert trajectories to improve robustness.

    This simulates the natural variation in human driving — two expert drivers
    navigating the same route will produce slightly different trajectories.
    Augmenting with noise prevents the model from overfitting to the exact
    waypoint positions in the training data.

    Args:
        trajectory: Expert trajectory, shape (32, 3) — [x, y, theta] per waypoint
        noise_std: Standard deviation of Gaussian noise to add

    Returns:
        Augmented trajectory with same shape

    Hints:
        Step 1: Generate Gaussian noise with torch.randn_like, scaled by noise_std
        Step 2: Add noise only to the x and y channels (indices 0 and 1), NOT theta
                (heading noise should be recomputed from the noisy x,y positions)
        Step 3: Recompute theta from the noisy x,y using torch.atan2 on the
                finite differences (gradient) of y and x
        Step 4: Clamp x and y to [-1, 1] to keep waypoints in valid range
    """
    # YOUR CODE HERE (approximately 6-8 lines)
    pass


def augment_mirror(image: torch.Tensor, instruction: torch.Tensor,
                   trajectory: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Mirror the scene horizontally — flip the image, negate the x-coordinates
    of the trajectory, and negate the heading angles.

    This effectively doubles the dataset by creating left-right mirror images.
    In a symmetric port layout, a trajectory going left is equally valid as
    one going right.

    Args:
        image: Port scene image, shape (3, 128, 128)
        instruction: Tokenized instruction, shape (32,) — left unchanged
        trajectory: Expert trajectory, shape (32, 3)

    Returns:
        Tuple of (flipped_image, instruction, flipped_trajectory)

    Hints:
        Step 1: Flip the image horizontally using torch.flip on the last dimension
        Step 2: Negate the x-coordinates (index 0) of the trajectory
        Step 3: Negate the theta values (index 2) of the trajectory
        Step 4: Return the instruction unchanged (text is not mirrored)
    """
    # YOUR CODE HERE (approximately 4-5 lines)
    pass

In [ ]:
# --- Verification for TODO 1 ---
test_traj = torch.randn(32, 3)
test_traj[:, :2] = test_traj[:, :2].clamp(-0.9, 0.9)

if augment_trajectory_noise is not None and augment_trajectory_noise(test_traj) is not None:
    aug_traj = augment_trajectory_noise(test_traj, noise_std=0.01)
    assert aug_traj.shape == (32, 3), f"Expected shape (32, 3), got {aug_traj.shape}"
    assert not torch.allclose(aug_traj[:, :2], test_traj[:, :2]), "Noise was not added to x,y"
    assert (aug_traj[:, :2].abs() <= 1.0).all(), "x,y values out of [-1, 1] range"
    print("[PASS] augment_trajectory_noise: shape, noise, and clamping correct")
else:
    print("[SKIP] augment_trajectory_noise not yet implemented")

# Test augment_mirror
test_img = torch.randn(3, 128, 128)
test_inst = torch.randint(0, 50, (32,))
test_traj2 = torch.randn(32, 3)

if augment_mirror is not None and augment_mirror(test_img, test_inst, test_traj2) is not None:
    f_img, f_inst, f_traj = augment_mirror(test_img, test_inst, test_traj2)
    assert f_img.shape == test_img.shape, "Image shape changed"
    assert torch.allclose(f_traj[:, 0], -test_traj2[:, 0]), "x not negated"
    assert torch.allclose(f_traj[:, 2], -test_traj2[:, 2]), "theta not negated"
    assert torch.allclose(f_inst, test_inst), "Instruction should be unchanged"
    print("[PASS] augment_mirror: flip and negation correct")
else:
    print("[SKIP] augment_mirror not yet implemented")

---

## 2. Exploratory Data Analysis

### 2.1 Port Layout and Sample Visualization

In [ ]:
env = train_dataset.env
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 1. Port layout
axes[0].imshow(env.grid, cmap='Oranges', origin='lower')
for name, (by, bx, ey, ex) in env.block_positions.items():
    axes[0].text(bx + (ex - bx) / 2, by - 2, f"Block {name}",
                 ha='center', fontsize=8, fontweight='bold')
axes[0].set_title("Port Yard Layout (Container Blocks)")
axes[0].set_xlabel("X (grid cells)")
axes[0].set_ylabel("Y (grid cells)")

# 2. Sample scene with trajectory
sample = train_dataset.data[0]
img = sample["image"].permute(1, 2, 0).numpy()
axes[1].imshow(img, origin='lower')
traj = sample["trajectory"].numpy()
sy, sx = sample["start"]
plot_x = traj[:, 0] * (PORT_SIZE / 2) + sx
plot_y = traj[:, 1] * (PORT_SIZE / 2) + sy
axes[1].plot(plot_x, plot_y, 'g-', linewidth=2, label='Expert trajectory')
axes[1].plot(plot_x[0], plot_y[0], 'go', markersize=8, label='Start')
axes[1].plot(plot_x[-1], plot_y[-1], 'r*', markersize=12, label='End')
axes[1].legend(fontsize=8)
axes[1].set_title("Sample Scene + Trajectory")

# 3. Trajectory length distribution
lengths = []
for d in train_dataset.data[:500]:
    t = d["trajectory"].numpy()
    dx = np.diff(t[:, 0])
    dy = np.diff(t[:, 1])
    length = np.sum(np.sqrt(dx**2 + dy**2))
    lengths.append(length)
axes[2].hist(lengths, bins=30, edgecolor='black', alpha=0.7)
axes[2].set_xlabel("Trajectory Length (normalized units)")
axes[2].set_ylabel("Count")
axes[2].set_title("Distribution of Trajectory Lengths")
axes[2].axvline(np.mean(lengths), color='red', linestyle='--', label=f'Mean: {np.mean(lengths):.2f}')
axes[2].legend()

plt.tight_layout()
plt.show()

### 2.2 Dataset Statistics

In [ ]:
print(f"Total training samples: {len(train_dataset)}")
print(f"Total validation samples: {len(val_dataset)}")
print(f"Trajectory shape: {train_dataset[0][3].shape}")
print(f"Trajectory value range: [{train_dataset[0][3].min():.3f}, {train_dataset[0][3].max():.3f}]")
print(f"Mean trajectory length: {np.mean(lengths):.3f}")
print(f"Std trajectory length: {np.std(lengths):.3f}")

### 2.3 TODO 2: EDA Deep Dive

In [ ]:
# TODO 2: Exploratory Data Analysis
#
# Analyze and plot the following (write code for each):
#
# 1. Plot a 2x3 grid showing 6 random training samples — each subplot should show
#    the port image with the expert trajectory overlaid. Title each with the
#    destination block/row.
#
# 2. Compute and plot the distribution of waypoint HEADINGS (theta values) across
#    the dataset. This tells you whether trajectories are predominantly straight,
#    turning left, or turning right. A balanced distribution is important for
#    training an unbiased model.
#
# 3. Plot a heatmap of trajectory ENDPOINTS across the port grid. This shows which
#    destinations are sampled most frequently. Ideally, all blocks should have
#    roughly equal representation.

# YOUR CODE HERE

**Thought questions** (answer in the space below):
- Q1: Are any trajectory heading directions underrepresented? How might this bias the model's turning behavior?
- Q2: If certain port blocks have more endpoint samples, would the model navigate to those blocks more accurately? What data balancing strategy would you use?
- Q3: How does the trajectory length distribution relate to the distance between blocks? Are short trajectories (nearby destinations) overrepresented?

---

## 3. Baseline Approaches

### 3.1 Straight-Line Baseline

The simplest baseline: draw a straight line from current position to destination, ignoring all obstacles.

In [ ]:
class StraightLineBaseline:
    """Predict a straight-line trajectory from current position to destination."""

    def predict(self, image: torch.Tensor, instruction: torch.Tensor,
                egomotion: torch.Tensor) -> torch.Tensor:
        """
        Extract destination from the image (channel 2 = destination marker)
        and plan a straight line to it.
        """
        # Find destination from image channel 2
        dest_channel = image[2]  # (128, 128)
        dest_mask = (dest_channel > 0.5)
        if dest_mask.sum() == 0:
            trajectory = torch.zeros(NUM_WAYPOINTS, 3)
            trajectory[:, 1] = torch.linspace(0, 0.3, NUM_WAYPOINTS)
            return trajectory

        ys, xs = torch.where(dest_mask)
        dest_y = ys.float().mean().item()
        dest_x = xs.float().mean().item()

        cy, cx = PORT_SIZE / 2, PORT_SIZE / 2
        traj_x = torch.linspace(0, (dest_x - cx) / (PORT_SIZE / 2), NUM_WAYPOINTS)
        traj_y = torch.linspace(0, (dest_y - cy) / (PORT_SIZE / 2), NUM_WAYPOINTS)
        theta = torch.atan2(traj_y[-1] - traj_y[0], traj_x[-1] - traj_x[0]).expand(NUM_WAYPOINTS)

        trajectory = torch.stack([traj_x, traj_y, theta], dim=-1)
        return trajectory

### 3.2 Evaluation Metrics

In [ ]:
def compute_metrics(pred_traj: torch.Tensor, gt_traj: torch.Tensor) -> Dict[str, float]:
    """Compute ADE, FDE, and smoothness metrics."""
    # Average Displacement Error
    displacement = torch.sqrt(((pred_traj[:, :2] - gt_traj[:, :2]) ** 2).sum(dim=-1))
    ade = displacement.mean().item()

    # Final Displacement Error
    fde = displacement[-1].item()

    # Smoothness (jerk = third derivative of position)
    dt = 0.1  # 10Hz
    velocity = torch.diff(pred_traj[:, :2], dim=0) / dt
    acceleration = torch.diff(velocity, dim=0) / dt
    jerk = torch.diff(acceleration, dim=0) / dt
    smoothness = torch.sqrt((jerk ** 2).sum(dim=-1)).mean().item()

    return {"ADE": ade, "FDE": fde, "Jerk": smoothness}

In [ ]:
# Evaluate straight-line baseline
baseline = StraightLineBaseline()
baseline_metrics = {"ADE": [], "FDE": [], "Jerk": []}

for i in range(min(200, len(val_dataset))):
    image, instruction, egomotion, gt_traj = val_dataset[i]
    pred = baseline.predict(image, instruction, egomotion)
    metrics = compute_metrics(pred, gt_traj)
    for k, v in metrics.items():
        baseline_metrics[k].append(v)

print("=== Straight-Line Baseline Results ===")
for k in baseline_metrics:
    vals = baseline_metrics[k]
    print(f"  {k}: {np.mean(vals):.4f} +/- {np.std(vals):.4f}")

### 3.3 TODO 3: Obstacle-Aware Baseline

In [ ]:
class ObstacleAwareBaseline:
    """
    A baseline that plans a trajectory toward the destination while avoiding
    obstacles visible in the image.

    This represents what a traditional (non-ML) planner would do: extract
    obstacle positions from the sensor data and plan around them using
    geometric rules. It still cannot understand language instructions, but
    it should produce collision-free trajectories.

    Args:
        safety_margin: Minimum distance (in grid cells) to maintain from obstacles

    Hints:
        Step 1: Extract the destination from image channel 2 (same as StraightLineBaseline)
        Step 2: Extract obstacle positions from image channel 0 (containers) and
                channel 1 (workers)
        Step 3: Generate a straight-line trajectory to the destination
        Step 4: For each waypoint, check if it is within safety_margin of any obstacle.
                If so, shift it perpendicular to the trajectory direction by safety_margin.
        Step 5: Smooth the adjusted trajectory using a simple moving average
        Step 6: Recompute heading angles from the smoothed x,y positions
    """

    def __init__(self, safety_margin: float = 3.0):
        self.safety_margin = safety_margin

    def predict(self, image: torch.Tensor, instruction: torch.Tensor,
                egomotion: torch.Tensor) -> torch.Tensor:
        # YOUR CODE HERE (approximately 25-35 lines)
        pass

In [ ]:
# --- Verification for TODO 3 ---
if ObstacleAwareBaseline().predict(val_dataset[0][0], val_dataset[0][1], val_dataset[0][2]) is not None:
    improved = ObstacleAwareBaseline()
    imp_metrics = {"ADE": [], "FDE": [], "Jerk": []}
    for i in range(min(200, len(val_dataset))):
        image, instruction, egomotion, gt_traj = val_dataset[i]
        pred = improved.predict(image, instruction, egomotion)
        assert pred.shape == (NUM_WAYPOINTS, 3), f"Wrong shape: {pred.shape}"
        m = compute_metrics(pred, gt_traj)
        for k, v in m.items():
            imp_metrics[k].append(v)

    print("=== Obstacle-Aware Baseline Results ===")
    for k in imp_metrics:
        vals = imp_metrics[k]
        print(f"  {k}: {np.mean(vals):.4f} +/- {np.std(vals):.4f}")
    print("\n(Compare with straight-line baseline above)")
else:
    print("[SKIP] ObstacleAwareBaseline not yet implemented")

---

## 4. VLA Model Architecture

### 4.1 Vision Encoder

A CNN that processes the 3-channel port scene image into a 256-dimensional feature vector. In a production VLA (like Alpamayo), this would be a Vision Transformer processing multi-camera inputs. We use a CNN here for training efficiency.

In [ ]:
class VisionEncoder(nn.Module):
    """
    Encode the port scene image into a feature vector.

    Input: (B, 3, 128, 128) — 3-channel port image
    Output: (B, 256) — scene feature vector
    """

    def __init__(self, feature_dim: int = 256):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(128, feature_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B = x.shape[0]
        h = self.conv(x).view(B, -1)
        return self.fc(h)

### 4.2 Text Encoder

An embedding layer + single-layer Transformer that converts tokenized dispatcher instructions into a 256-dimensional feature vector. In a production VLA, this would be a pre-trained LLM like LLaMA.

In [ ]:
class TextEncoder(nn.Module):
    """
    Encode dispatcher instructions into a feature vector.

    Input: (B, max_len) — tokenized instruction
    Output: (B, 256) — instruction feature vector
    """

    def __init__(self, vocab_size: int, embed_dim: int = 128,
                 feature_dim: int = 256, max_len: int = 32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=4, dim_feedforward=256,
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)
        self.fc = nn.Linear(embed_dim, feature_dim)

    def forward(self, tokens: torch.Tensor) -> torch.Tensor:
        B, L = tokens.shape
        positions = torch.arange(L, device=tokens.device).unsqueeze(0).expand(B, -1)
        x = self.embedding(tokens) + self.pos_embedding(positions)
        pad_mask = (tokens == 0)
        x = self.transformer(x, src_key_padding_mask=pad_mask)
        mask = (~pad_mask).float().unsqueeze(-1)
        x = (x * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)
        return self.fc(x)

### 4.3 Diffusion Trajectory Decoder

The core of our VLA: a denoiser network that predicts the noise added to a trajectory, conditioned on visual and linguistic features.

In [ ]:
class SinusoidalTimestepEmbedding(nn.Module):
    """Sinusoidal positional encoding for diffusion timestep."""

    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=t.device, dtype=torch.float32) * -emb)
        emb = t.float().unsqueeze(-1) * emb.unsqueeze(0)
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        return emb


class TrajectoryDenoiser(nn.Module):
    """
    Predicts noise added to a trajectory, conditioned on timestep and scene/language features.

    Input:
        noisy_traj: (B, 32, 3) — noisy trajectory
        timestep: (B,) — diffusion timestep [0, T)
        condition: (B, 512) — concatenated vision + text features

    Output: (B, 32, 3) — predicted noise
    """

    def __init__(self, traj_dim: int = 32 * 3, cond_dim: int = 512,
                 hidden_dim: int = 512, time_dim: int = 128):
        super().__init__()
        self.time_embed = SinusoidalTimestepEmbedding(time_dim)
        self.time_proj = nn.Linear(time_dim, hidden_dim)
        self.cond_proj = nn.Linear(cond_dim, hidden_dim)

        self.net = nn.Sequential(
            nn.Linear(traj_dim + hidden_dim + hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, traj_dim),
        )

    def forward(self, noisy_traj: torch.Tensor, timestep: torch.Tensor,
                condition: torch.Tensor) -> torch.Tensor:
        B = noisy_traj.shape[0]
        traj_flat = noisy_traj.view(B, -1)  # (B, 96)
        t_emb = self.time_proj(self.time_embed(timestep))  # (B, hidden)
        c_emb = self.cond_proj(condition)  # (B, hidden)

        x = torch.cat([traj_flat, t_emb, c_emb], dim=-1)
        noise_pred = self.net(x)
        return noise_pred.view(B, 32, 3)

### 4.4 Noise Schedule

In [ ]:
class NoiseSchedule:
    """Linear noise schedule for diffusion."""

    def __init__(self, num_steps: int = 100, beta_start: float = 1e-4, beta_end: float = 0.02):
        self.num_steps = num_steps
        self.beta = torch.linspace(beta_start, beta_end, num_steps)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        self.sqrt_alpha_bar = torch.sqrt(self.alpha_bar)
        self.sqrt_one_minus_alpha_bar = torch.sqrt(1.0 - self.alpha_bar)

    def to(self, device):
        self.beta = self.beta.to(device)
        self.alpha = self.alpha.to(device)
        self.alpha_bar = self.alpha_bar.to(device)
        self.sqrt_alpha_bar = self.sqrt_alpha_bar.to(device)
        self.sqrt_one_minus_alpha_bar = self.sqrt_one_minus_alpha_bar.to(device)
        return self


schedule = NoiseSchedule(num_steps=100)

### 4.5 TODO 4: Assemble the Full VLA Model

This is the core exercise. Assemble VisionEncoder + TextEncoder + TrajectoryDenoiser into a complete VLA model with both training forward pass and inference sampling.

In [ ]:
class PortVLA(nn.Module):
    """
    A simplified Vision-Language-Action model for port terminal tractors.

    This model combines:
    1. A VisionEncoder to process the port scene image
    2. A TextEncoder to process the dispatcher instruction
    3. A TrajectoryDenoiser (diffusion decoder) to generate trajectories

    The forward pass for TRAINING:
    - Encode the image and instruction into feature vectors
    - Concatenate them into a conditioning vector
    - Sample a random timestep and add noise to the expert trajectory
    - Predict the noise using the denoiser
    - Return the predicted noise and actual noise (for loss computation)

    The forward pass for INFERENCE (sampling):
    - Encode image and instruction
    - Start from pure noise
    - Iteratively denoise using the reverse diffusion process
    - Return the generated trajectory

    Hints:
        __init__:
            Step 1: Create VisionEncoder(feature_dim=vision_dim)
            Step 2: Create TextEncoder(vocab_size, feature_dim=text_dim)
            Step 3: Create TrajectoryDenoiser(cond_dim=vision_dim + text_dim)
            Step 4: Create NoiseSchedule(num_steps=num_diffusion_steps)

        forward (training mode):
            Step 1: Encode image with vision encoder -> (B, vision_dim)
            Step 2: Encode instruction with text encoder -> (B, text_dim)
            Step 3: Concatenate vision and text features -> (B, vision_dim + text_dim)
            Step 4: Sample random timesteps: torch.randint(0, T, (B,))
            Step 5: Sample random noise: torch.randn_like(trajectory)
            Step 6: Create noisy trajectory using forward diffusion:
                    noisy = sqrt_alpha_bar[t] * trajectory + sqrt_one_minus_alpha_bar[t] * noise
                    (Reshape schedule values for broadcasting: [..., None, None])
            Step 7: Predict noise using denoiser(noisy, timesteps, condition)
            Step 8: Return (predicted_noise, actual_noise)

        sample (inference mode):
            Step 1: Encode image and instruction, concatenate
            Step 2: Start from pure noise: x = torch.randn(B, 32, 3)
            Step 3: Loop from t = T-1 down to 0:
                    a. Predict noise: eps = denoiser(x, t, condition)
                    b. Compute the denoised mean using the DDPM reverse formula
                    c. If t > 0, add scaled random noise
            Step 4: Return the final denoised trajectory
    """

    def __init__(self, vocab_size: int, vision_dim: int = 256,
                 text_dim: int = 256, num_diffusion_steps: int = 100):
        super().__init__()
        # YOUR CODE HERE (approximately 5-7 lines)
        pass

    def forward(self, image: torch.Tensor, instruction: torch.Tensor,
                trajectory: torch.Tensor):
        """Training forward pass. Returns (predicted_noise, actual_noise)."""
        # YOUR CODE HERE (approximately 10-12 lines)
        pass

    @torch.no_grad()
    def sample(self, image: torch.Tensor, instruction: torch.Tensor,
               num_samples: int = 1) -> torch.Tensor:
        """Generate trajectory samples via reverse diffusion."""
        # YOUR CODE HERE (approximately 15-20 lines)
        pass

In [ ]:
# --- Verification for TODO 4 ---
try:
    model = PortVLA(vocab_size=train_dataset.vocab_size).to(device)

    # Test training forward pass
    test_img = torch.randn(4, 3, 128, 128).to(device)
    test_inst = torch.randint(0, train_dataset.vocab_size, (4, 32)).to(device)
    test_traj = torch.randn(4, 32, 3).to(device)

    pred_noise, actual_noise = model(test_img, test_inst, test_traj)
    assert pred_noise.shape == (4, 32, 3), f"Wrong noise shape: {pred_noise.shape}"
    assert actual_noise.shape == (4, 32, 3), f"Wrong noise shape: {actual_noise.shape}"
    print("[PASS] Training forward pass: correct shapes")

    # Test sampling
    sampled = model.sample(test_img[:1], test_inst[:1], num_samples=1)
    assert sampled.shape == (1, 32, 3), f"Wrong sample shape: {sampled.shape}"
    print("[PASS] Sampling: correct shape")

    total_params = sum(p.numel() for p in model.parameters())
    print(f"\nTotal parameters: {total_params:,}")
except Exception as e:
    print(f"[FAIL] {e}")
    print("Implement PortVLA in TODO 4 above, then re-run this cell.")

---

## 5. Training

### 5.1 Training Configuration

In [ ]:
BATCH_SIZE = 64
LEARNING_RATE = 3e-4
NUM_EPOCHS = 30
WEIGHT_DECAY = 1e-4
WARMUP_STEPS = 200

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

### 5.2 TODO 5: Training Loop

In [ ]:
def train_vla(model, train_loader, val_loader, num_epochs, lr, weight_decay, warmup_steps):
    """
    Train the VLA model using the diffusion denoising objective.

    The training objective is simple: predict the noise that was added to the
    expert trajectory. The loss is MSE between predicted and actual noise.

    We use AdamW (not Adam) because weight decay helps prevent overfitting on
    our relatively small synthetic dataset. The cosine learning rate schedule
    with warmup provides stable early training (warmup prevents large initial
    gradient updates) and graceful convergence (cosine decay avoids the abrupt
    drops of step schedules).

    Args:
        model: PortVLA model
        train_loader: Training data loader
        val_loader: Validation data loader
        num_epochs: Number of training epochs
        lr: Peak learning rate
        weight_decay: AdamW weight decay
        warmup_steps: Number of linear warmup steps

    Returns:
        Dict with 'train_losses' and 'val_losses' lists

    Hints:
        Step 1: Create AdamW optimizer with the given lr and weight_decay
        Step 2: Create a learning rate scheduler — use
                torch.optim.lr_scheduler.CosineAnnealingLR (T_max = num_epochs * len(train_loader))
                For warmup: manually scale LR for the first warmup_steps
        Step 3: For each epoch:
            a. Set model to train mode
            b. For each batch (image, instruction, egomotion, trajectory):
                - Move all tensors to device
                - Forward pass: pred_noise, actual_noise = model(image, instruction, trajectory)
                - Compute MSE loss: F.mse_loss(pred_noise, actual_noise)
                - Backward pass and optimizer step
                - Apply learning rate warmup if within warmup_steps
                - Step the scheduler
            c. Compute validation loss (model.eval(), no gradients)
            d. Print epoch stats: train loss, val loss, learning rate
        Step 4: Return the loss histories
    """
    # YOUR CODE HERE (approximately 35-45 lines)
    pass

In [ ]:
# --- Run Training ---
# (Uncomment after implementing TODO 5)

# history = train_vla(model, train_loader, val_loader,
#                     num_epochs=NUM_EPOCHS, lr=LEARNING_RATE,
#                     weight_decay=WEIGHT_DECAY, warmup_steps=WARMUP_STEPS)

### 5.3 Training Curves

In [ ]:
# --- Visualize Training Curves ---
# (Uncomment after training completes)

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
#
# ax1.plot(history['train_losses'], label='Train Loss', alpha=0.7)
# ax1.plot(history['val_losses'], label='Val Loss', alpha=0.7)
# ax1.set_xlabel('Epoch')
# ax1.set_ylabel('MSE Loss')
# ax1.set_title('Training and Validation Loss')
# ax1.legend()
# ax1.grid(True, alpha=0.3)
#
# ax2.plot(history.get('lrs', []), alpha=0.7)
# ax2.set_xlabel('Step')
# ax2.set_ylabel('Learning Rate')
# ax2.set_title('Learning Rate Schedule')
# ax2.grid(True, alpha=0.3)
#
# plt.tight_layout()
# plt.show()

---

## 6. Evaluation

### 6.1 Quantitative Evaluation

In [ ]:
def evaluate_model(model, val_loader, num_samples=5):
    """
    Evaluate the VLA model on the validation set.
    For each sample, generate num_samples trajectories and pick the best (lowest ADE).
    This leverages the diffusion model's multimodality.
    """
    model.eval()
    all_metrics = {"ADE": [], "FDE": [], "Jerk": []}

    with torch.no_grad():
        for batch_idx, (image, instruction, egomotion, gt_traj) in enumerate(val_loader):
            if batch_idx >= 10:  # Evaluate on 10 batches
                break
            image = image.to(device)
            instruction = instruction.to(device)
            gt_traj = gt_traj.to(device)

            for i in range(min(4, image.shape[0])):
                best_ade = float('inf')
                best_pred = None

                for _ in range(num_samples):
                    pred = model.sample(image[i:i+1], instruction[i:i+1])
                    m = compute_metrics(pred[0].cpu(), gt_traj[i].cpu())
                    if m["ADE"] < best_ade:
                        best_ade = m["ADE"]
                        best_pred = pred[0].cpu()
                        best_metrics = m

                for k, v in best_metrics.items():
                    all_metrics[k].append(v)

    print("=== VLA Model Evaluation (best of 5 samples) ===")
    for k, vals in all_metrics.items():
        print(f"  {k}: {np.mean(vals):.4f} +/- {np.std(vals):.4f}")

    return all_metrics


# Uncomment after training:
# vla_metrics = evaluate_model(model, val_loader)

### 6.2 TODO 6: Comparative Evaluation

In [ ]:
# TODO 6: Generate a Comparative Evaluation
#
# After training, complete the following evaluation:
#
# 1. Run evaluate_model() on your trained VLA
# 2. Collect baseline metrics from Section 3
# 3. Create a bar chart comparing ADE, FDE, and Jerk for:
#    - Straight-Line Baseline
#    - Obstacle-Aware Baseline (your TODO 3)
#    - VLA Model
#    Use grouped bars with error bars showing +/- 1 std deviation.
#    Include a horizontal dashed line at the target threshold for each metric.
#
# 4. Create a 2x3 figure showing 6 validation samples:
#    - Each subplot: port image + ground-truth trajectory (green) + VLA prediction (red)
#    - Title each with ADE for that sample
#
# 5. Answer these thought questions:
#    Q1: Where does the VLA outperform the baselines most significantly? Why?
#    Q2: Are there scenarios where the straight-line baseline is actually better
#        than the VLA? Why might this happen?
#    Q3: How would you expect performance to change if we increased the dataset
#        from 5,000 to 50,000 samples? Which metric would improve most?

# YOUR CODE HERE

---

## 7. Error Analysis

### 7.1 TODO 7: Systematic Error Analysis

In [ ]:
# TODO 7: Systematic Error Analysis
#
# After training and evaluation, perform a systematic error analysis:
#
# 1. Identify the 20 validation samples with the HIGHEST ADE (worst predictions).
#    For each, visualize the scene image, ground-truth trajectory, and VLA prediction.
#    Look for patterns: are failures concentrated in specific port areas, long
#    distances, scenes with many workers, or particular instruction types?
#
# 2. Categorize failures into at least 3 categories. Suggested categories:
#    - "Long distance": destination is far away, trajectory accumulates error
#    - "Obstacle avoidance": trajectory clips an obstacle
#    - "Instruction misunderstanding": trajectory goes to wrong destination
#    - "Multimodal confusion": trajectory is a valid path but to a different
#      plausible destination (evidence of mode averaging)
#
# 3. For each category:
#    - Count how many of the top-20 failures fall into it
#    - Propose a specific fix (more data, architectural change, loss modification)
#    - Explain WHY that fix addresses the root cause
#
# 4. Compute the collision rate: what percentage of VLA-generated trajectories
#    pass within 1 grid cell of an obstacle (container or worker)?

# YOUR CODE HERE

---

## 8. Deployment Considerations

### 8.1 Inference Latency Profiling

In [ ]:
def profile_inference(model, image, instruction, num_runs=20):
    """Profile inference latency by component."""
    model.eval()
    image = image.to(device)
    instruction = instruction.to(device)

    # Warm up
    for _ in range(3):
        _ = model.sample(image, instruction)

    # Profile
    times = []
    for _ in range(num_runs):
        torch.cuda.synchronize() if device.type == 'cuda' else None
        start = time.perf_counter()
        _ = model.sample(image, instruction)
        torch.cuda.synchronize() if device.type == 'cuda' else None
        times.append((time.perf_counter() - start) * 1000)

    times = np.array(times)
    print(f"=== Inference Latency ({num_runs} runs) ===")
    print(f"  Mean: {times.mean():.1f} ms")
    print(f"  Std:  {times.std():.1f} ms")
    print(f"  P50:  {np.percentile(times, 50):.1f} ms")
    print(f"  P95:  {np.percentile(times, 95):.1f} ms")
    print(f"  P99:  {np.percentile(times, 99):.1f} ms")
    return times

# Uncomment after training:
# test_img = val_dataset[0][0].unsqueeze(0)
# test_inst = val_dataset[0][1].unsqueeze(0)
# latencies = profile_inference(model, test_img, test_inst)

### 8.2 TODO 8: DDIM Accelerated Sampling

In [ ]:
def ddim_sample(model, image, instruction, num_ddim_steps=10):
    """
    Implement DDIM (Denoising Diffusion Implicit Models) sampling to reduce
    the number of denoising steps from 100 to 10.

    DDIM is deterministic — it skips timesteps by taking larger denoising jumps.
    This is critical for deployment: 100 denoising steps at ~2ms each = 200ms,
    which barely meets the latency budget. 10 DDIM steps = ~20ms, leaving
    headroom for vision and text encoding.

    Args:
        model: Trained PortVLA model
        image: (1, 3, 128, 128) input image
        instruction: (1, 32) tokenized instruction
        num_ddim_steps: Number of DDIM sampling steps (default 10)

    Returns:
        trajectory: (1, 32, 3) generated trajectory

    Hints:
        Step 1: Compute the DDIM timestep subsequence — evenly space num_ddim_steps
                timesteps across [0, T). Use torch.linspace(0, T-1, num_ddim_steps).long()
        Step 2: Encode image and instruction to get the condition vector
        Step 3: Start from noise x = torch.randn(1, 32, 3)
        Step 4: Loop through timesteps in REVERSE order. At each step:
                a. Predict noise: eps = denoiser(x, t, condition)
                b. Compute predicted x0: x0_pred = (x - sqrt(1-alpha_bar_t) * eps) / sqrt(alpha_bar_t)
                c. If not the last step, compute x at the previous timestep:
                   x = sqrt(alpha_bar_prev) * x0_pred + sqrt(1 - alpha_bar_prev) * eps
                   (This is the DDIM update — no stochastic noise added)
                d. If last step, x = x0_pred
        Step 5: Return x
    """
    # YOUR CODE HERE (approximately 20-25 lines)
    pass

---

## 9. Ethical and Regulatory Analysis

### 9.1 TODO 9: Ethical Impact Assessment

Write a brief (300-500 word) ethical impact assessment for deploying the VLA-based autonomous terminal tractor system in a markdown cell below. Address:

1. **Worker Safety**: False positives (nuisance stops) vs. false negatives (missed detections). Which way should the system be biased, and why?
2. **Labor Displacement**: 140 drivers reduced to 20 supervisors. What is the company's ethical obligation?
3. **Accountability**: If an accident occurs, who is liable? How does the reasoning trace help?
4. **Bias and Fairness**: Training data from one terminal — how might this bias generalization?
5. **Regulatory Compliance**: List 3 specific regulations (MTSA, OSHA, ISO 3691) and one requirement from each.

---

## Summary

In this notebook, you built a simplified Vision-Language-Action model for autonomous port terminal tractors. The core components you implemented:

1. **Synthetic port environment** with container blocks, corridors, workers, and dispatcher instructions
2. **Vision encoder** (CNN) for processing port scene images
3. **Text encoder** (Transformer) for processing natural language instructions
4. **Diffusion trajectory decoder** for generating smooth, multimodal trajectories
5. **Training pipeline** using the denoising objective (behavioral cloning via diffusion)
6. **Evaluation** comparing the VLA against straight-line and obstacle-aware baselines
7. **Error analysis** to systematically categorize and address failure modes
8. **DDIM sampling** for deployment-friendly inference latency

The architecture mirrors what NVIDIA's Alpamayo uses at production scale — a vision encoder feeds into a VLM backbone which conditions a diffusion decoder. The key insight is that diffusion-based action generation handles **multimodal trajectory distributions** (multiple valid paths) without the mode-averaging problem of regression-based approaches.

For further reading on production deployment, system design, and scaling considerations, see **Section 4** of the accompanying case study document (`case_study.pdf`).